<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    TensorBoard Callback Overview
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
#
# ==========================================================
# Lesson: Understanding tensorboard callback
#         Synthetic Data Example
# ==========================================================
# ## Learning Objectives
# This example will enable you to:
# 1. Understand the tensorboard callback
# 2. Setup the environment for using tensorboard
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2025
# =======================================================================

In [ ]:
# ============================================================================ #
#                         Environment Path Configuration                       #
# ============================================================================ #
#
# Purpose:
#   Configure the system PATH to use Python executables from the active virtual 
#   environment instead of global installations.
#
# Usage:
#   1. First verify if configuration is needed by running: !which python
#   2. If the output shows the global Python installation rather than your 
#      virtual environment, execute this configuration block
#
# Note:
#   This configuration is typically only needed for JupyterLab Desktop or 
#   similar standalone installations. Web-based JupyterLab or properly 
#   configured environments should not require this adjustment.
# ============================================================================ #

import os
import sys

env_path = os.path.dirname(sys.executable)
os.environ['PATH'] = f"{env_path}:{os.environ['PATH']}"

In [1]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #

import shutil
from datetime import datetime
import io


# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  

# import tensorflow
import tensorflow as tf

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2024-12-31

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
sklearn   : 1.5.1
keras     : 3.6.0
matplotlib: 3.9.2
tensorflow: 2.16.2
seaborn   : 0.13.2
numpy     : 1.26.4
pandas    : 2.2.2



## Introduction

**TensorBoard** is a visualization toolkit for `TensorFlow` that allows you to monitor and analyze your machine learning model's training process. The **TensorBoard** callback in Keras provides an easy way to log various metrics and data during model training.

## Environment Setup with Tensorboard

### Required Imports
```python
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import datetime
```

### Creating the Basic Callback
```python
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2
)
```

### Using the Callback
```python
model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_val, y_val),
    callbacks=[tensorboard_callback]
)
```

## Key Parameters

### Essential Parameters
- `log_dir`: Directory where logs will be saved
- `histogram_freq`: Frequency (in epochs) for computing weight histograms
- `write_graph`: Whether to visualize the model graph
- `write_images`: Whether to write model weights as images
- `update_freq`: When to write logs ('batch', 'epoch', or integer)
- `profile_batch`: Which batch to profile for computation characteristics

### Additional Parameters
- `embeddings_freq`: Frequency for saving embeddings
- `embeddings_metadata`: Dictionary mapping layer names to metadata files
- `embeddings_layer_names`: List of layers to visualize embeddings for

## Features and Capabilities

### 1. Metrics Visualization
Automatically logs:
- Loss values
- Metrics specified in `model.compile()`
- Learning rates
- Gradient norms

### 2. Model Graph Visualization
- Shows model architecture
- Displays tensor shapes
- Shows operations and data flow

### 3. Distribution and Histogram Tracking
- Weight distributions
- Activation distributions
- Gradient distributions
- Custom histogram data

### 4. Image Logging
```python
with file_writer.as_default():
    tf.summary.image("Training data", img_tensor, step=epoch)
```

### 5. Text Logging
```python
with file_writer.as_default():
    tf.summary.text('hyperparameters', str(hyperparameters), step=0)
```

# TensorBoard Best Practices Guide

This guide outlines current best practices for using TensorBoard with modern TensorFlow/Keras implementations.

## Log Directory Management

### Structured Log Directories

```python
import datetime
import os

def create_log_dir(model_name, base_dir='logs'):
    """
    Create a structured log directory with timestamp.
    
    Args:
        model_name (str): Name of the model
        base_dir (str): Base directory for logs
    
    Returns:
        str: Path to the log directory
    """
    timestamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    log_dir = os.path.join(base_dir, 'fit', model_name, timestamp)
    os.makedirs(log_dir, exist_ok=True)
    return log_dir
```

### Log Directory Cleanup

```python
import shutil

def cleanup_logs(log_dir='logs'):
    """
    Safely remove old log directories.
    
    Args:
        log_dir (str): Directory to clean up
    """
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir, ignore_errors=True)
```

## TensorBoard Callback Configuration

### Basic Configuration

```python
import tensorflow as tf

def create_tensorboard_callback(log_dir):
    """
    Create a TensorBoard callback with recommended settings.
    
    Args:
        log_dir (str): Directory for storing logs
    
    Returns:
        tf.keras.callbacks.TensorBoard: Configured TensorBoard callback
    """
    return tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,         # Enable histogram computation
        write_graph=True,         # Log model graph
        write_images=False,       # Skip image logging for efficiency
        update_freq='epoch',      # Log at end of each epoch
        profile_batch=0,          # Disable profiling by default
    )
```

### Profiling Configuration

```python
def create_profiling_callback(log_dir, profile_batch='2,5'):
    """
    Create a TensorBoard callback with profiling enabled.
    
    Args:
        log_dir (str): Directory for storing logs
        profile_batch (str): Batches to profile (start,end)
    
    Returns:
        tf.keras.callbacks.TensorBoard: TensorBoard callback with profiling
    """
    return tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        profile_batch=profile_batch
    )
```

## Custom Metrics Logging

### Modern Approach (Recommended)

```python
class MetricsCallback(tf.keras.callbacks.Callback):
    """
    Callback for logging custom metrics using modern TensorFlow features.
    """
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        
        # Log custom metrics directly using tf.summary
        tf.summary.scalar('custom_metric', data=logs.get('loss', 0) * 2, step=epoch)
```

## Usage Example

```python
# Setup logging
model_name = 'my_model'
log_dir = create_log_dir(model_name)

# Create callbacks
tensorboard_callback = create_tensorboard_callback(log_dir)
metrics_callback = MetricsCallback()

# Train model with callbacks
model.fit(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_val, y_val),
    callbacks=[
        tensorboard_callback,
        metrics_callback
    ]
)
```

## Best Practices Summary

1. **Structured Logging**:
   - Use timestamp-based directories
   - Separate logs by model and run
   - Clean up old logs when necessary

2. **Resource Management**:
   - Disable profiling unless needed
   - Log histograms sparingly
   - Avoid logging images unless required

3. **Custom Metrics**:
   - Use `tf.summary` directly for custom metrics
   - Avoid creating separate file writers
   - Leverage built-in Keras metrics when possible

4. **Profiling**:
   - Enable profiling only for specific batches
   - Use profile_batch parameter judiciously
   - Consider performance impact when profiling

Remember to launch TensorBoard after training:

```python
%load_ext tensorboard
%tensorboard --logdir logs
```

## Best Practices

### 1. Log Directory Structure
```python
log_dir = f"logs/fit/{model_name}/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
```

### 2. Separate Training and Validation Writers
```python
train_log_dir = 'logs/gradient_tape/train'
test_log_dir = 'logs/gradient_tape/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
```

### 3. Profile Selectively
```python
tensorboard_callback = TensorBoard(profile_batch=(1, 100))
```

### 4. Clean Up Old Logs
```python
import shutil
shutil.rmtree('logs/fit/', ignore_errors=True)
```

## Custom Logging

### Creating a Custom Callback
```python
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(CustomCallback, self).__init__()
        self.writer = tf.summary.create_file_writer(log_dir)
    
    def on_epoch_end(self, epoch, logs={}):
        with self.writer.as_default():
            tf.summary.scalar('custom_metric', data=logs['loss'] * 2, step=epoch)
```

# Model Logging in TensorBoard

TensorBoard provides powerful capabilities for monitoring and analyzing your model during training. This guide covers best practices for logging various aspects of your model using modern TensorFlow/Keras approaches.

## Common Logging Scenarios

### 1. Learning Rate Monitoring

To track learning rate changes during training, you can create a simple callback:

```python
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate
        if hasattr(lr, 'numpy'):
            lr = lr.numpy()
        tf.summary.scalar('learning_rate', lr, step=epoch)
```

### 2. Custom Metrics

For tracking custom metrics, you have two approaches:

#### A. Using Keras Metrics API (Recommended)
```python
class CustomMetric(tf.keras.metrics.Metric):
    def __init__(self, name='custom_metric', **kwargs):
        super().__init__(name=name, **kwargs)
        self.value = self.add_weight(name='value', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        # Your custom calculation here
        calculated_value = tf.reduce_mean(y_pred)
        self.value.assign(calculated_value)
    
    def result(self):
        return self.value
    
    def reset_state(self):
        self.value.assign(0.0)

# Use in model compilation
model.compile(metrics=[CustomMetric()])
```

#### B. Using Callback for Complex Cases
```python
class MetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Calculate your custom metric
        value = compute_custom_metric()
        tf.summary.scalar('custom_metric', value, step=epoch)
```

### 3. Weight and Gradient Monitoring

TensorBoard can automatically track weights and gradients with proper configuration:

```python
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=1,  # Enable histogram computation every epoch
    write_graph=True,  # Log model graph
    update_freq='epoch'
)
```

For custom weight statistics:
```python
class WeightLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        for layer in self.model.layers:
            if layer.weights:
                for weight in layer.weights:
                    name = f'{layer.name}/{weight.name}'
                    tf.summary.histogram(name, weight, step=epoch)
                    # Log additional statistics
                    tf.summary.scalar(
                        f'{name}_mean', 
                        tf.reduce_mean(weight), 
                        step=epoch
                    )
```

## Complete Logging Setup

Here's a complete setup combining all logging features:

```python
def create_model_logger(model_name, base_log_dir='logs'):
    """Create a complete logging setup for model training."""
    # Create timestamped log directory
    timestamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    log_dir = os.path.join(base_log_dir, model_name, timestamp)
    os.makedirs(log_dir, exist_ok=True)
    
    # Create callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True,
        update_freq='epoch'
    )
    
    callbacks = [
        tensorboard_callback,
        LearningRateLogger(),
        WeightLogger()
    ]
    
    return log_dir, callbacks
```

## Usage Example

Here's how to use the logging setup in practice:

```python
# Create and compile model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup logging
log_dir, callbacks = create_model_logger('example_model')

# Compile with custom metric
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', CustomMetric()]
)

# Train with logging
model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_val, y_val),
    callbacks=callbacks
)
```

## Viewing Logs in TensorBoard

Launch TensorBoard in a Jupyter notebook:
```python
%load_ext tensorboard
%tensorboard --logdir logs
```

Or from the command line:
```bash
tensorboard --logdir logs
```

## Best Practices

1. **Organized Log Directories**
   - Use timestamp-based directories
   - Group related runs under model-specific directories
   - Clean up old logs periodically

2. **Resource Management**
   - Log histograms sparingly (they can be expensive)
   - Use appropriate update frequencies
   - Consider disk space for long training runs

3. **Custom Metrics**
   - Prefer Keras Metrics API over callbacks when possible
   - Use meaningful names for metrics
   - Consider computational cost of custom calculations

4. **Performance Monitoring**
   - Enable profiling selectively
   - Monitor resource usage
   - Log only necessary metrics

This setup provides a comprehensive view of your model's training process, including learning rates, custom metrics, weight distributions, and general training progress.

### Common Custom Logging Scenarios
1. **Learning Rate Changes**
```python
tf.summary.scalar('learning_rate', optimizer.lr, step=epoch)
```

2. **Custom Metrics**
```python
tf.summary.scalar('custom_loss', calculated_value, step=batch)
```

3. **Model Weights**
```python
tf.summary.histogram('layer_weights', layer.weights[0], step=epoch)
```

## Launching TensorBoard

### Command Line
```bash
tensorboard --logdir logs/fit
```

### In Jupyter Notebook

We can use **TensorBoard** directly within Jupyter notebooks using IPython magic commands. Here are the the steps of doing so: 

1. First, load the `TensorBoard` extension:

```python
%load_ext tensorboard
```

2. Launch TensorBoard in the notebook:

```python
%tensorboard --logdir logs/fit
```
This command will create an interactive TensorBoard interface directly in your notebook cell


- **Some additional useful variations**:

    ```python
    # Specify port if default is taken
    %tensorboard --logdir logs/fit --port=6007

    # Launch with specific host
    %tensorboard --logdir logs/fit --host=0.0.0.0
    ```

3. Check Tensorboard Help Page

```python
%tensorboard --helpfull
```

### Benefits of using TensorBoard in Jupyter:

- No need to switch between browser windows
- Easier to share notebooks with TensorBoard visualizations
- Can have multiple TensorBoard instances in different cells
- Better integration with your experimental workflow

### TensorBoard Interface in Jupyter

The TensorBoard interface in Jupyter has all the same features as the standalone version, including:
- Scalars
- Graphs
- Distributions
- Histograms
- Images
- Text logs
- Profiler data

## Common Issues and Solutions

### 1. Missing Data
- Ensure proper log directory structure
- Verify write permissions
- Check callback is in the callbacks list

### 2. Performance Issues
- Limit histogram computation frequency
- Use selective profiling
- Clean old logs regularly

### 3. Memory Issues
- Reduce update frequency
- Limit number of histograms
- Use appropriate batch sizes for profiling

## Additional Tips

1. **Organizing Experiments**
   - Use meaningful experiment names
   - Create separate directories for different model versions
   - Include timestamp in log directory names

2. **Debugging**
   - Use debug mode in TensorBoard
   - Check file writer scopes
   - Verify log file creation

3. **Performance Optimization**
   - Use appropriate update frequencies
   - Limit histogram computation
   - Profile selectively

## Resources

- TensorFlow Documentation: [TensorBoard Guide](https://www.tensorflow.org/tensorboard)
- GitHub Repository: [TensorBoard](https://github.com/tensorflow/tensorboard)
- Tutorial: [TensorBoard in TensorFlow 2](https://www.tensorflow.org/tensorboard/get_started)